# Project: API to PostgreSQL  
ดึงข้อมูล Crypto จาก CoinGecko API → เก็บ CSV → Load เข้าฐานข้อมูล PostgreSQL

In [12]:
pip install requests pandas psycopg2-binary dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd

url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)
df.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,104911.000,2084931473987,1,2084939341518,2.590046e+10,105213.000,...,1.987542e+07,2.100000e+07,111814.00,-6.18346,2025-05-22T18:41:28.492Z,67.810000,1.545995e+05,2013-07-06T00:00:00.000Z,None,2025-06-07T08:30:50.144Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2489.410,300533501400,2,300533501400,1.456779e+10,2519.250,...,1.207218e+08,NaN,4878.26,-48.94412,2021-11-10T14:24:19.604Z,0.432979,5.751333e+05,2015-10-20T00:00:00.000Z,"{'times': 30.701961161084753, 'currency': 'btc...",2025-06-07T08:30:39.558Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.001,154544914275,3,154544914275,3.821034e+10,1.001,...,1.544683e+11,NaN,1.32,-24.38176,2018-07-24T00:00:00.000Z,0.572521,7.475367e+01,2015-03-02T00:00:00.000Z,None,2025-06-07T08:30:48.101Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.180,128398890905,4,218254651493,1.545200e+09,2.190,...,9.998606e+10,1.000000e+11,3.40,-35.75602,2018-01-07T00:00:00.000Z,0.002686,8.117817e+04,2014-05-22T00:00:00.000Z,None,2025-06-07T08:30:50.295Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,648.460,94600429472,5,94600429472,6.533362e+08,651.370,...,1.458876e+08,2.000000e+08,788.84,-17.76528,2024-12-04T10:35:25.220Z,0.039818,1.629084e+06,2017-10-19T00:00:00.000Z,None,2025-06-07T08:30:51.160Z


In [4]:
df.to_csv("crypto_data.csv", index=False)

In [5]:
import psycopg2
from dotenv import load_dotenv
import os

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    database=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS crypto_prices (
        id TEXT,
        name TEXT,
        current_price FLOAT,
        market_cap BIGINT
    )
""")
conn.commit()

In [6]:
df = pd.read_csv("crypto_data.csv")

In [7]:
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO crypto_prices (id, name, current_price, market_cap)
        VALUES (%s, %s, %s, %s)
    """, (row['id'], row['name'], row['current_price'], row['market_cap']))
conn.commit()

In [8]:
cur.execute("SELECT * FROM crypto_prices LIMIT 5")
rows = cur.fetchall()
for r in rows:
    print(r)

('bitcoin', 'Bitcoin', 103686.0, 2061096941111)
('ethereum', 'Ethereum', 2482.8, 299719748521)
('tether', 'Tether', 1.0, 153889992225)
('ripple', 'XRP', 2.15, 126438820367)
('binancecoin', 'BNB', 643.45, 93907460859)


In [9]:
cur.close()
conn.close()